#### Notebook looks at the inner-workings of the Teacheable agent, and understand how it solves the problem of retaining its teachings

Teachable Agent required ChromaDB, which requires a long list of dependencies. So moving the notebook to Linux environment, and explore from there. 

As this notebook requires the linux environment, it will test the AssistantAgent execution with local "Codellama2" or "llama2" models

Steps Followed:

- Setting up the Gradio endpoints in Kali Server, and test the generation end point in this notebook. (This did not Work.. So parked it)

- Create the config_list to only work on the Teachable agent and undrestand it